# Module 3 - Exercise 1: Decision Trees

<a href="https://colab.research.google.com/github/jumpingsphinx/jumpingsphinx.github.io/blob/main/notebooks/module3-trees/exercise1-decision-trees.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Objectives

By the end of this exercise, you will be able to:

- Build decision trees from scratch
- Understand and calculate Gini impurity and entropy
- Implement splitting criteria for optimal tree construction
- Visualize decision trees and their decision boundaries
- Analyze overfitting and apply pruning techniques
- Apply decision trees to both classification and regression problems

## Prerequisites

- Completion of Module 1 and 2
- Understanding of classification and regression
- Familiarity with information theory concepts

## Setup

Run this cell first to import required libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, fetch_california_housing, make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

print("NumPy version:", np.__version__)
print("Setup complete!")

---

## Part 1: Understanding Splitting Criteria

### Background

Decision trees select splits that maximize information gain by minimizing impurity. Two common impurity measures are:

**Gini Impurity:**

$$\text{Gini} = 1 - \sum_{i=1}^{C} p_i^2$$

**Entropy (Information):**

$$\text{Entropy} = -\sum_{i=1}^{C} p_i \log_2(p_i)$$

Where $p_i$ is the proportion of samples belonging to class $i$ and $C$ is the number of classes.

### Exercise 1.1: Calculate Gini Impurity

**Task:** Implement a function to calculate Gini impurity for a set of labels.

In [ ]:
def gini_impurity(labels):
    """
    Calculate Gini impurity for a set of labels.
    
    Parameters:
    -----------
    labels : np.ndarray
        Array of class labels
    
    Returns:
    --------
    float
        Gini impurity value between 0 (pure) and 0.5 (binary case, most impure)
    """
    # Your code here
    unique_classes, counts = np.unique(labels, return_counts=True)
    proportions = counts / len(labels)
    gini = 1 - np.sum(proportions ** 2)
# Test cases
# Pure node: all same class
pure_labels = np.array([0, 0, 0, 0, 0])
print(f"Pure node Gini: {gini_impurity(pure_labels):.4f}")  # Expected: 0.0

# Maximally impure node: equal split (binary case)
impure_labels = np.array([0, 0, 1, 1])
print(f"50/50 split Gini: {gini_impurity(impure_labels):.4f}")  # Expected: 0.5

# Mixed node
mixed_labels = np.array([0, 0, 0, 1, 1])
print(f"Mixed node Gini: {gini_impurity(mixed_labels):.4f}")  # Expected: 0.48

# Three classes
three_class = np.array([0, 0, 1, 1, 2, 2])
print(f"Three classes Gini: {gini_impurity(three_class):.4f}")  # Expected: 0.6667

assert gini_impurity(pure_labels) == 0.0, "Pure node should have Gini = 0"
assert np.isclose(gini_impurity(impure_labels), 0.5), "50/50 split should have Gini = 0.5"
print("\n✓ Gini impurity implemented correctly!")

### Exercise 1.2: Calculate Entropy

**Task:** Implement a function to calculate entropy for a set of labels.

In [ ]:
def entropy(labels):
    """
    Calculate entropy for a set of labels.
    
    Parameters:
    -----------
    labels : np.ndarray
        Array of class labels
    
    Returns:
    --------
    float
        Entropy value (0 = pure, higher = more impure)
    """
    # Your code here
    unique_classes, counts = np.unique(labels, return_counts=True)
    proportions = counts / len(labels)
    
    # Avoid log(0)
    proportions = proportions[proportions > 0]
    entropy_val = -np.sum(proportions * np.log2(proportions))
# Test cases
print(f"Pure node Entropy: {entropy(pure_labels):.4f}")  # Expected: 0.0
print(f"50/50 split Entropy: {entropy(impure_labels):.4f}")  # Expected: 1.0
print(f"Mixed node Entropy: {entropy(mixed_labels):.4f}")  # Expected: ~0.971
print(f"Three classes Entropy: {entropy(three_class):.4f}")  # Expected: ~1.585

assert entropy(pure_labels) == 0.0, "Pure node should have entropy = 0"
assert np.isclose(entropy(impure_labels), 1.0), "50/50 split should have entropy = 1.0"
print("\n✓ Entropy implemented correctly!")

### Exercise 1.3: Calculate Information Gain

**Task:** Implement information gain calculation.

Information Gain measures how much a split reduces impurity:

$$\text{InfoGain} = \text{Impurity}_{\text{parent}} - \sum_{\text{child}} \frac{N_{\text{child}}}{N_{\text{parent}}} \times \text{Impurity}_{\text{child}}$$

In [ ]:
def information_gain(parent_labels, left_labels, right_labels, criterion='gini'):
    """
    Calculate information gain from a split.
    
    Parameters:
    -----------
    parent_labels : labels before split
    left_labels : labels in left child
    right_labels : labels in right child
    criterion : 'gini' or 'entropy'
    
    Returns:
    --------
    float
        Information gain value
    """
    # Your code here
    parent_impurity = impurity_func(parent_labels)
    
    # Calculate weighted average impurity of children
    n_parent = len(parent_labels)
    n_left = len(left_labels)
    n_right = len(right_labels)
    
    weighted_child_impurity = (n_left / n_parent) * impurity_func(left_labels) +                               (n_right / n_parent) * impurity_func(right_labels)
    
    gain = parent_impurity - weighted_child_impurity
# Test: Split [0,0,0,1,1,1] -> [0,0,0] and [1,1,1]
parent = np.array([0, 0, 0, 1, 1, 1])
left = np.array([0, 0, 0])
right = np.array([1, 1, 1])

gain_gini = information_gain(parent, left, right, 'gini')
gain_entropy = information_gain(parent, left, right, 'entropy')

print(f"Information Gain (Gini): {gain_gini:.4f}")  # Expected: 0.5
print(f"Information Gain (Entropy): {gain_entropy:.4f}")  # Expected: 1.0

# Perfect split should have high gain
assert gain_gini == 0.5, "Perfect split should have Gini gain = 0.5"
assert np.isclose(gain_entropy, 1.0), "Perfect split should have Entropy gain = 1.0"
print("\n✓ Information gain implemented correctly!")

---

## Part 2: Implementing Decision Tree from Scratch

### Background

A decision tree recursively:
1. Finds the best split (feature and threshold)
2. Divides data into left and right children
3. Repeats until stopping criteria (max depth, min samples, pure node)

### Exercise 2.1: Implement Tree Node

**Task:** Create a simple decision tree node structure.

In [ ]:
class TreeNode:
    """
    A node in the decision tree.
    """
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        """
        Parameters:
        -----------
        feature : int
            Feature index for splitting (None for leaf)
        threshold : float
            Threshold value for splitting (None for leaf)
        left : TreeNode
            Left child node
        right : TreeNode
            Right child node
        value : int/float
            Predicted class (for leaf nodes)
        """
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        """Check if node is a leaf."""
        return self.value is not None

print("TreeNode class created!")

### Exercise 2.2: Implement Simple Decision Tree

**Task:** Implement a basic decision tree classifier with limited depth.

In [ ]:
class SimpleDecisionTree:
    """
    A simple decision tree classifier implementation.
    """
    def __init__(self, max_depth=5, min_samples_split=2, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None
    
    def fit(self, X, y):
        """Build the decision tree."""
        self.root = self._grow_tree(X, y, depth=0)
        return self
    
    def _grow_tree(self, X, y, depth):
        """
        Recursively grow the tree.
        
        Returns:
        --------
        TreeNode
        """
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        # Stopping criteria
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            n_classes == 1):
            # Create leaf node with majority class
            leaf_value = np.argmax(np.bincount(y))
            return TreeNode(value=leaf_value)
        
        # Find best split
        best_feature, best_threshold = self._best_split(X, y)
        
        if best_feature is None:
            # No good split found
            leaf_value = np.argmax(np.bincount(y))
            return TreeNode(value=leaf_value)
        
        # Split data
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices
        
        # Recursively build left and right subtrees
        left_child = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        
        return TreeNode(feature=best_feature, threshold=best_threshold,
                       left=left_child, right=right_child)
    
    def _best_split(self, X, y):
        """
        Find the best feature and threshold to split on.
        
        Returns:
        --------
        best_feature : int or None
        best_threshold : float or None
        """
        # Your code here
        n_samples, n_features = X.shape
        best_gain = -1
        best_feature = None
        best_threshold = None
        
        for feature in range(n_features):
            # Get unique values for this feature
            thresholds = np.unique(X[:, feature])
            
            for threshold in thresholds:
                # Split data
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                
                # Skip if split is empty
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                # Calculate information gain
                gain = information_gain(y, y[left_mask], y[right_mask], self.criterion)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        # Your code here
print("SimpleDecisionTree class implemented!")

### Exercise 2.3: Test Your Implementation

**Task:** Test your decision tree on a simple dataset.

In [ ]:
# Create simple dataset
X_simple = np.array([
    [2, 3],
    [3, 4],
    [4, 5],
    [1, 2],
    [2, 2],
    [7, 8],
    [8, 9],
    [9, 10],
    [6, 7],
    [7, 7]
])
y_simple = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# Train your tree
tree = SimpleDecisionTree(max_depth=3)
tree.fit(X_simple, y_simple)

# Make predictions
y_pred = tree.predict(X_simple)

# Calculate accuracy
accuracy = np.mean(y_pred == y_simple)
print(f"Training Accuracy: {accuracy:.2%}")

# Visualize decision boundary
def plot_decision_boundary(X, y, model, title="Decision Boundary"):
    h = 0.1
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='black', cmap='viridis', s=100)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.colorbar(label='Class')
    plt.show()

plot_decision_boundary(X_simple, y_simple, tree, "Your Decision Tree")

assert accuracy >= 0.8, "Should achieve at least 80% accuracy"
print("\n✓ Your decision tree works!")

---

## Part 3: Scikit-learn Decision Trees

### Background

Scikit-learn provides optimized implementations with many features:
- Multiple splitting criteria (gini, entropy, log_loss)
- Pruning strategies
- Support for categorical features
- Feature importance calculation

### Exercise 3.1: Train on Iris Dataset

**Task:** Use DecisionTreeClassifier on the Iris dataset.

In [ ]:
# Load Iris dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print("Iris Dataset:")
print(f"Samples: {X_iris.shape[0]}")
print(f"Features: {iris.feature_names}")
print(f"Classes: {iris.target_names}")
print()

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42
)

# Your code here: Create and train a DecisionTreeClassifier
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)
print("Decision Tree Classifier on Iris:")
print(f"Train Accuracy: {train_acc:.2%}")
print(f"Test Accuracy:  {test_acc:.2%}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=iris.target_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix:")
print(cm)

assert test_acc >= 0.85, "Should achieve at least 85% accuracy on Iris"
print("\n✓ Successfully trained on Iris dataset!")

### Exercise 3.2: Feature Importance

**Task:** Analyze which features are most important for classification.

In [ ]:
# Get feature importances
importances = clf.feature_importances_

# Create DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'Feature': iris.feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)
print()

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(range(len(importances)), importances)
plt.yticks(range(len(importances)), iris.feature_names)
plt.xlabel('Importance')
plt.title('Feature Importance in Decision Tree')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Most important feature: {iris.feature_names[np.argmax(importances)]}")

---

## Part 4: Visualization

### Exercise 4.1: Visualize Tree Structure

**Task:** Create a visual representation of the decision tree.

In [ ]:
# Train a smaller tree for better visualization
clf_small = DecisionTreeClassifier(max_depth=3, random_state=42)
clf_small.fit(X_train, y_train)

# Visualize the tree
plt.figure(figsize=(20, 10))
plot_tree(clf_small, 
          feature_names=iris.feature_names,
          class_names=iris.target_names,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Decision Tree Structure (max_depth=3)', fontsize=16)
plt.show()

print("Tree Characteristics:")
print(f"Max depth: {clf_small.get_depth()}")
print(f"Number of leaves: {clf_small.get_n_leaves()}")
print(f"Number of nodes: {clf_small.tree_.node_count}")

### Exercise 4.2: Decision Boundaries (2D)

**Task:** Visualize decision boundaries using 2 features.

In [ ]:
# Use only 2 features for 2D visualization
# Let's use petal length and petal width (features 2 and 3)
X_2d = X_iris[:, [2, 3]]
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(
    X_2d, y_iris, test_size=0.3, random_state=42
)

# Train tree on 2D data
clf_2d = DecisionTreeClassifier(max_depth=4, random_state=42)
clf_2d.fit(X_train_2d, y_train_2d)

# Plot decision boundary
def plot_decision_boundary_multiclass(X, y, model, feature_names, class_names):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(12, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
    
    # Plot training points
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, 
                         edgecolors='black', s=100, cmap='viridis')
    
    plt.xlabel(feature_names[0], fontsize=12)
    plt.ylabel(feature_names[1], fontsize=12)
    plt.title('Decision Tree Decision Boundaries', fontsize=14)
    plt.colorbar(scatter, ticks=range(len(class_names)), label='Class')
    plt.legend(handles=scatter.legend_elements()[0], 
              labels=class_names, loc='upper left')
    plt.show()

plot_decision_boundary_multiclass(
    X_train_2d, y_train_2d, clf_2d,
    ['Petal Length', 'Petal Width'],
    iris.target_names
)

test_acc_2d = clf_2d.score(X_test_2d, y_test_2d)
print(f"\nTest Accuracy (2D): {test_acc_2d:.2%}")

---

## Part 5: Overfitting Analysis

### Background

Decision trees can easily overfit by growing too deep. The `max_depth` hyperparameter controls complexity.

### Exercise 5.1: Vary max_depth

**Task:** Train trees with different depths and compare performance.

In [ ]:
# Test different max_depth values
depths = range(1, 21)
train_scores = []
test_scores = []

for depth in depths:
    # Your code here: Train tree with this depth

# Your code here: Create and train a DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calculate accuracies
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Optimal max_depth: {optimal_depth}")
print(f"Best test accuracy: {best_test_score:.2%}")
print(f"\nOverfitting analysis:")
print(f"At max_depth=1: Train={train_scores[0]:.2%}, Test={test_scores[0]:.2%} (Underfitting)")
print(f"At max_depth={optimal_depth}: Train={train_scores[optimal_depth-1]:.2%}, Test={test_scores[optimal_depth-1]:.2%} (Good)")
print(f"At max_depth=20: Train={train_scores[-1]:.2%}, Test={test_scores[-1]:.2%} (Potential Overfitting)")

### Exercise 5.2: Cross-Validation

**Task:** Use cross-validation for more robust evaluation.

In [ ]:
# Test different depths with cross-validation
depths_cv = [1, 2, 3, 4, 5, 10, 15, 20]
cv_scores_mean = []
cv_scores_std = []

for depth in depths_cv:
    clf_cv = DecisionTreeClassifier(max_depth=depth, random_state=42)
    
    # Perform 5-fold cross-validation
    scores = cross_val_score(clf_cv, X_iris, y_iris, cv=5, scoring='accuracy')
    
    cv_scores_mean.append(scores.mean())
    cv_scores_std.append(scores.std())

# Plot with error bars
plt.figure(figsize=(12, 6))
plt.errorbar(depths_cv, cv_scores_mean, yerr=cv_scores_std, 
             marker='o', capsize=5, linewidth=2, markersize=8)
plt.xlabel('Max Depth', fontsize=12)
plt.ylabel('Cross-Validation Accuracy', fontsize=12)
plt.title('5-Fold Cross-Validation Scores vs Max Depth', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

best_depth_cv = depths_cv[np.argmax(cv_scores_mean)]
best_cv_score = np.max(cv_scores_mean)
print(f"\nBest depth (CV): {best_depth_cv}")
print(f"Best CV accuracy: {best_cv_score:.2%} (+/- {cv_scores_std[np.argmax(cv_scores_mean)]:.2%})")

---

## Part 6: Regression Trees

### Background

Decision trees can also solve regression problems by predicting continuous values. Leaf nodes contain the mean of training samples in that region.

### Exercise 6.1: California Housing Dataset

**Task:** Apply DecisionTreeRegressor to predict house prices.

In [ ]:
# Load California Housing dataset
housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target

print("California Housing Dataset:")
print(f"Samples: {X_housing.shape[0]}")
print(f"Features: {housing.feature_names}")
print(f"Target: Median house value (in $100,000s)")
print()

# Split data
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

# Your code here: Create and train a DecisionTreeRegressor
    # Your code here: Train tree with this depth
    clf_depth = DecisionTreeClassifier(max_depth=depth, random_state=42)
    clf_depth.fit(X_train, y_train)
    
    # Calculate scores
    train_score = clf_depth.score(X_train, y_train)
    test_score = clf_depth.score(X_test, y_test)
print("Decision Tree Regressor Performance:")
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE:  {test_mse:.4f}")
print(f"Test RMSE: {np.sqrt(test_mse):.4f}")
print(f"Test MAE:  {test_mae:.4f}")
print(f"Train R²:  {train_r2:.4f}")
print(f"Test R²:   {test_r2:.4f}")

assert test_r2 >= 0.5, "Should achieve at least 0.5 R² on housing data"
print("\n✓ Regression tree trained successfully!")

### Exercise 6.2: Regression Predictions Visualization

**Task:** Visualize actual vs predicted values.

In [ ]:
# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Predicted vs Actual
axes[0].scatter(y_test_h, y_test_pred_h, alpha=0.5)
axes[0].plot([y_test_h.min(), y_test_h.max()], 
             [y_test_h.min(), y_test_h.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Price ($100k)', fontsize=12)
axes[0].set_ylabel('Predicted Price ($100k)', fontsize=12)
axes[0].set_title('Predicted vs Actual House Prices', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals
residuals_h = y_test_h - y_test_pred_h
axes[1].scatter(y_test_pred_h, residuals_h, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Price ($100k)', fontsize=12)
axes[1].set_ylabel('Residuals', fontsize=12)
axes[1].set_title('Residual Plot', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean of residuals: {residuals_h.mean():.4f}")
print(f"Std of residuals: {residuals_h.std():.4f}")

### Exercise 6.3: Regression Tree Depth Analysis

**Task:** Analyze overfitting in regression trees.

In [ ]:
# Test different max_depth values for regression
depths_reg = range(1, 21)
train_r2_scores = []
test_r2_scores = []
train_rmse_scores = []
test_rmse_scores = []

for depth in depths_reg:
    reg = DecisionTreeRegressor(max_depth=depth, random_state=42)
    reg.fit(X_train_h, y_train_h)
    
    y_train_pred = reg.predict(X_train_h)
    y_test_pred = reg.predict(X_test_h)
    
    train_r2_scores.append(r2_score(y_train_h, y_train_pred))
    test_r2_scores.append(r2_score(y_test_h, y_test_pred))
    train_rmse_scores.append(np.sqrt(mean_squared_error(y_train_h, y_train_pred)))
    test_rmse_scores.append(np.sqrt(mean_squared_error(y_test_h, y_test_pred)))

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# R² plot
axes[0].plot(depths_reg, train_r2_scores, 'o-', label='Training R²', linewidth=2)
axes[0].plot(depths_reg, test_r2_scores, 's-', label='Test R²', linewidth=2)
axes[0].set_xlabel('Max Depth', fontsize=12)
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('R² vs Max Depth', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RMSE plot
axes[1].plot(depths_reg, train_rmse_scores, 'o-', label='Training RMSE', linewidth=2)
axes[1].plot(depths_reg, test_rmse_scores, 's-', label='Test RMSE', linewidth=2)
axes[1].set_xlabel('Max Depth', fontsize=12)
axes[1].set_ylabel('RMSE', fontsize=12)
axes[1].set_title('RMSE vs Max Depth', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

optimal_depth_reg = depths_reg[np.argmax(test_r2_scores)]
print(f"\nOptimal max_depth for regression: {optimal_depth_reg}")
print(f"Best test R²: {np.max(test_r2_scores):.4f}")
print(f"Best test RMSE: {test_rmse_scores[np.argmax(test_r2_scores)]:.4f}")

---

## Part 7: Real-World Application - Titanic Dataset

### Background

Predict survival on the Titanic using decision trees. We'll create a Titanic-like dataset using sklearn's make_classification.

### Exercise 7.1: Create and Explore Titanic-like Data

**Task:** Generate a binary classification dataset.

In [ ]:
# Create Titanic-like dataset
# Features: Age, Fare, Class, Sex, Siblings, Parents
X_titanic, y_titanic = make_classification(
    n_samples=891,  # Similar to Titanic dataset size
    n_features=6,
    n_informative=4,
    n_redundant=1,
    n_classes=2,
    weights=[0.62, 0.38],  # ~38% survival rate
    random_state=42
)

# Create DataFrame for easier manipulation
titanic_features = ['Age', 'Fare', 'Pclass', 'Sex', 'Siblings', 'Parents']
df_titanic = pd.DataFrame(X_titanic, columns=titanic_features)
df_titanic['Survived'] = y_titanic

print("Titanic-like Dataset:")
print(f"Total passengers: {len(df_titanic)}")
print(f"Survival rate: {y_titanic.mean():.2%}")
print(f"\nFirst few rows:")
print(df_titanic.head())
print(f"\nDataset info:")
print(df_titanic.describe())

### Exercise 7.2: Train and Evaluate Decision Tree

**Task:** Build a decision tree to predict survival.

In [ ]:
# Split data
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_titanic, y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

# Your code here: Train a decision tree with appropriate hyperparameters
clf_titanic = DecisionTreeClassifier(
    max_depth=,  # Choose a good value
    min_samples_split=,
    min_samples_leaf=,
    random_state=42
)
clf_titanic.fit(X_train_t, y_train_t)

# Predictions
y_train_pred_t = clf_titanic.predict(X_train_t)
y_test_pred_t = clf_titanic.predict(X_test_t)

# Evaluation
train_acc_t = accuracy_score(y_train_t, y_train_pred_t)
test_acc_t = accuracy_score(y_test_t, y_test_pred_t)

print("Titanic Survival Prediction:")
print(f"Train Accuracy: {train_acc_t:.2%}")
print(f"Test Accuracy:  {test_acc_t:.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test_t, y_test_pred_t, 
                          target_names=['Did not survive', 'Survived']))

# Confusion Matrix
cm_t = confusion_matrix(y_test_t, y_test_pred_t)
print("\nConfusion Matrix:")
print("                Predicted")
print("              Not Survive  Survive")
print(f"Actual Not Survive    {cm_t[0,0]}         {cm_t[0,1]}")
print(f"Actual Survive        {cm_t[1,0]}         {cm_t[1,1]}")

assert test_acc_t >= 0.7, "Should achieve at least 70% accuracy"
print("\n✓ Titanic prediction model trained!")

### Exercise 7.3: Feature Importance Analysis

**Task:** Determine which factors most influenced survival.

In [ ]:
# Feature importance
importances_t = clf_titanic.feature_importances_

# Create DataFrame
feature_importance_t = pd.DataFrame({
    'Feature': titanic_features,
    'Importance': importances_t
}).sort_values('Importance', ascending=False)

print("Feature Importance for Titanic Survival:")
print(feature_importance_t)
print()

# Plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(importances_t)), 
         feature_importance_t['Importance'].values,
         color='steelblue')
plt.yticks(range(len(importances_t)), 
           feature_importance_t['Feature'].values)
plt.xlabel('Importance', fontsize=12)
plt.title('Feature Importance for Titanic Survival Prediction', fontsize=14)
plt.grid(True, alpha=0.3, axis='x')
plt.show()

print(f"Most important factor: {feature_importance_t.iloc[0]['Feature']}")

### Exercise 7.4: Visualize Titanic Decision Tree

**Task:** Create a visual representation of the survival decision tree.

In [ ]:
# Visualize the tree
plt.figure(figsize=(20, 12))
plot_tree(clf_titanic,
          feature_names=titanic_features,
          class_names=['Did not survive', 'Survived'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Titanic Survival Decision Tree', fontsize=16)
plt.show()

print(f"Tree depth: {clf_titanic.get_depth()}")
print(f"Number of leaves: {clf_titanic.get_n_leaves()}")

---

## Challenge Problems (Optional)

### Challenge 1: Pruning with min_samples_leaf

Implement post-pruning by experimenting with `min_samples_leaf` parameter.

In [ ]:
# Your code here
# Test min_samples_leaf values: 1, 5, 10, 20, 50
# Plot how tree complexity and accuracy change

min_samples_values = [1, 5, 10, 20, 50]

# TODO: Implement pruning analysis

print("Challenge 1: Analyze the effect of min_samples_leaf!")

### Challenge 2: Cost-Complexity Pruning

Use `ccp_alpha` parameter for cost-complexity pruning.

In [ ]:
# Your code here
# Hint: Use clf.cost_complexity_pruning_path() to get alpha values

print("Challenge 2: Implement cost-complexity pruning!")

### Challenge 3: Comparison with Other Algorithms

Compare decision tree performance with logistic regression and k-NN on the Iris dataset.

In [ ]:
# Your code here
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# TODO: Compare DecisionTree, LogisticRegression, and KNN

print("Challenge 3: Compare multiple algorithms!")

### Challenge 4: Ensemble of Stumps

Create an ensemble by training multiple decision stumps (depth=1) on different feature subsets.

In [ ]:
# Your code here
# Train multiple trees with max_depth=1 on different features
# Combine predictions by majority vote

print("Challenge 4: Build an ensemble of decision stumps!")

---

## Reflection Questions

1. **When would you prefer Gini impurity over Entropy?**
   - Consider computational efficiency and behavior differences
   - Gini is faster to compute (no logarithm)
   - Entropy may isolate one class in its own branch more often

2. **What are the main advantages of decision trees?**
   - Interpretability: Easy to understand and visualize
   - No feature scaling needed
   - Can handle both numerical and categorical data
   - Non-parametric (no assumptions about data distribution)
   - Captures non-linear relationships

3. **What are the main disadvantages?**
   - Prone to overfitting (especially deep trees)
   - Unstable: small changes in data can result in very different trees
   - Greedy algorithm: may not find globally optimal tree
   - Biased towards features with more levels

4. **How do you prevent overfitting in decision trees?**
   - Limit max_depth
   - Set min_samples_split and min_samples_leaf
   - Use pruning techniques (cost-complexity)
   - Ensemble methods (Random Forests, Gradient Boosting)

5. **When should you use decision trees vs linear models?**
   - Decision trees: Non-linear relationships, feature interactions, interpretability needed
   - Linear models: Linear relationships, high-dimensional sparse data, need for probabilistic outputs

---

## Summary

In this exercise, you learned:

✓ How to calculate Gini impurity and Entropy manually

✓ The concept of information gain for split selection

✓ Implementation of a basic decision tree from scratch

✓ Using scikit-learn's DecisionTreeClassifier and DecisionTreeRegressor

✓ Visualization of tree structures and decision boundaries

✓ Analysis of overfitting through hyperparameter tuning

✓ Feature importance analysis

✓ Application to both classification and regression tasks

✓ Cross-validation for robust model evaluation

**Next Steps:**

- Complete Exercise 2 on Random Forests
- Review the [Decision Trees lesson](https://jumpingsphinx.github.io/module3-trees/01-decision-trees/)
- Experiment with different datasets and hyperparameters
- Study ensemble methods that combine multiple trees

---

**Need help?** Check the solution notebook or open an issue on [GitHub](https://github.com/jumpingsphinx/jumpingsphinx.github.io/issues).